In [27]:
import sys
import os
import re
import pandas as pd
import numpy as np
from PIL import Image
from PIL import ImageDraw  
from PIL import ImageFont
from moviepy.editor import *
import moviepy.audio.fx.all as afx


class psychoCards:
    def __init__(self,xlsFn,title,machine):         
        if machine=='dzxc':
            self.dir='i:/大智小超/心理'
            self.font_dir='C:\Windows\Fonts'
            self.default_font='msyh.ttc'
        elif machine=='mi':
            self.dir='/home/jack/data/大智小超/心理'
            self.font_dir='/home/jack/files/FONTS'
            self.default_font='msyh.ttf'
        else:
            sys.exit('不能识别该机器，退出。') 
            
        print('正在查看是否存在“{}”文件夹……'.format(title))
        dirOn=os.path.exists(os.path.join(self.dir,title))  
        if not dirOn:
            print('不存在，正在创建……',end='')
            os.makedirs(os.path.join(self.dir,title))
            print('完成')
        else:
            print('存在')
            
        self.xlsFn=xlsFn
        self.title=title   
        
        bgm=os.path.join(self.dir,self.title,self.title+'.mp3')
        if os.path.exists(bgm):
            self.bgm=AudioFileClip(bgm).set_start(1)  
        else:
#             print('请先在文件夹中放入录音文件，然后再次运行代码。')
            sys.exit('请先在文件夹中放入录音文件，然后再次运行代码。')
    
    def makeList(self):
        fn=os.path.join(self.dir,self.xlsFn)
        ctns=pd.read_excel(fn)
#         print(ctns['标题二'],ctns['内容'],ctns['详细'])
        lst=np.array(ctns.loc[ctns['标题二']==self.title]['标题二']).tolist()
        lst.extend(ctns.loc[ctns['标题二']==self.title]['内容'])
#         lst.extend(ctns.loc[ctns['标题二']==self.title]['详细'])
#         print(lst)

        return lst
    
    def makePics(self,bg):
        print('开始根据文字输出图像……',end='')
        txts=self.makeList()
        title=txts[0]
        txt=txts[1].split('\n')
        img=Image.open(os.path.join(self.dir,bg))
        
        font_title = ImageFont.truetype(os.path.join(self.font_dir,self.default_font),30)
        font_txt = ImageFont.truetype(os.path.join(self.font_dir,self.default_font),20)
        draw = ImageDraw.Draw(img)            
        draw.text((40,int(img.size[1]*0.2)), title, fill = '#6AB34A',font=font_title)
        
        out=os.path.join(self.dir,self.title,'_img00.jpg')
        img.save(out)
        
        n=0
        for t in txt:
            img=Image.open(os.path.join(self.dir,self.title,'_img'+str(n).zfill(2)+'.jpg'))
            draw = ImageDraw.Draw(img)
            draw.fontmode='1'
            draw.text((40,int(img.size[1]*0.3+n*32)), t, fill = '#6AB34A',font=font_txt)
            img.save(os.path.join(self.dir,self.title,'_img'+str(n+1).zfill(2)+'.jpg'),quality=95)
            n+=1
        print('完成')
        
    def makeMovie(self,bg):
        self.makePics(bg)
        lst=self.makeList()
        drtns=[]        
        s=0
        f=0
        total_c=len(lst[1])
        total_time=self.bgm.duration
        spd=total_time/total_c
        
        txt=[lst[0]]
        txt.extend(lst[1].split('\n'))
#         print(txt)
        
        drtns=[]
        for t in txt:
            drtns.append(len(t)*spd)        
        drtns[0]=1
#         print(drtns)
        print('正在生成影片……')
        pics=self.re_name()
        clips=[]
        k=0
        ptn=r'_img.*jpg'
        for pic in pics:
            if re.match(ptn,pic.lower()) and pic.lower()[-3:]=='jpg':
#                 print(pic,txt[k],f,drtns[k])
                _img=ImageClip(os.path.join(self.dir,self.title,pic)).set_fps(25).set_start(f).set_duration(drtns[k]).resize((480,640))
                f=f+drtns[k]
                k+=1         
                
                clips.append(_img)
        finalclip = CompositeVideoClip(clips)
        
        
        
        # 添加语音
        empty_1s=AudioFileClip(os.path.join(self.dir,'empty1s.mp3')).set_start(0)
        
#         print(BGM.duration)
        final_audio = CompositeAudioClip([empty_1s,self.bgm])
        mix=finalclip.set_audio(final_audio)
        
        out=os.path.join(self.dir,self.title,self.title+'.mp4')
        mix.write_videofile(out)
        
        print('完成影片')
        
    def makeMovie_ppt(self):
        lst=self.makeList()
        drtns=[]        
        s=0
        f=0
        total_c=len(lst[1])
        total_time=self.bgm.duration
        spd=total_time/total_c
        
        txt=[lst[0]]
        txt.extend(lst[1].split('\n'))
#         print(txt)
        
        drtns=[]
        for t in txt:
            drtns.append(len(t)*spd)        
        drtns[0]=1
#         print(drtns)
        print('正在生成影片……')
        pics=self.re_name()
#         print(pics)
        clips=[]
        k=0
        ptn=r'幻灯片.*jpg'
        for pic in pics:
            if re.match(ptn,pic.lower()) and pic.lower()[-3:]=='jpg':
#                 print(pic,txt[k],f,drtns[k])
                _img=ImageClip(os.path.join(self.dir,self.title,pic)).set_fps(25).set_start(f).set_duration(drtns[k]).resize((480,640))
                f=f+drtns[k]
                k+=1         
                
                clips.append(_img)
        finalclip = CompositeVideoClip(clips)
        
        
        
        # 添加语音
        empty_1s=AudioFileClip(os.path.join(self.dir,'empty1s.mp3')).set_start(0)
        
#         print(BGM.duration)
        final_audio = CompositeAudioClip([empty_1s,self.bgm])
        mix=finalclip.set_audio(final_audio)
        
        out=os.path.join(self.dir,self.title,self.title+'_from_ppt.mp4')
        mix.write_videofile(out)
        
        print('完成影片')
    
    def re_name(self):
        print('重命名文件以排序……')
        old=os.path.join(self.dir,self.title)
        fn_list=os.listdir(old)
        ptn=r'幻灯片.*jpg'
        n=len(fn_list)
        m=0
        for fn in fn_list:
            q=re.match(ptn,fn.lower())
            if q and len(fn)<9:
                new=os.path.join(self.dir,self.title,fn[0:3]+'0'+fn[3:])
                os.rename(os.path.join(old,fn),new)
                m+=1
                
            if m>0:
                p='该文件夹内共有{0}个文件，重命名{1}个。'.format(n,m)
            else:
                p='该文件夹内共有{0}个文件，没有符合重命名条件的文件。'.format(n)
        print(p)
        new_list=os.listdir(old)
        new_list.sort()
#         print(new_list)
        return new_list
        
if __name__=='__main__':
    mycard=psychoCards('正面管教.xlsx','分清孩子的愿望和需要','dzxc')
#     mycard.makeMovie_ppt()
#     mycard.makePics('bg01.jpg')
    mycard.makeMovie('bg01.jpg')
    

正在查看是否存在“分清孩子的愿望和需要”文件夹……
存在
开始根据文字输出图像……完成
正在生成影片……
重命名文件以排序……
该文件夹内共有13个文件，没有符合重命名条件的文件。


chunk:   0%|                                                                         | 0/695 [00:00<?, ?it/s, now=None]

Moviepy - Building video i:/大智小超/心理\分清孩子的愿望和需要\分清孩子的愿望和需要.mp4.
MoviePy - Writing audio in 分清孩子的愿望和需要TEMP_MPY_wvf_snd.mp3


t:   0%|                                                                             | 0/731 [00:00<?, ?it/s, now=None]

MoviePy - Done.
Moviepy - Writing video i:/大智小超/心理\分清孩子的愿望和需要\分清孩子的愿望和需要.mp4



Moviepy - Done !
Moviepy - video ready i:/大智小超/心理\分清孩子的愿望和需要\分清孩子的愿望和需要.mp4
完成影片


In [25]:
# -*- coding:utf-8 -*-
#
#   author: iflytek
#
#  本demo测试时运行的环境为：Windows + Python3.7
#  本demo测试成功运行时所安装的第三方库及其版本如下：
#   cffi==1.12.3
#   gevent==1.4.0
#   greenlet==0.4.15
#   pycparser==2.19
#   six==1.12.0
#   websocket==0.2.1
#   websocket-client==0.56.0
#   合成小语种需要传输小语种文本、使用小语种发音人vcn、tte=unicode以及修改文本编码方式
#  错误码链接：https://www.xfyun.cn/document/error-code （code返回错误码时必看）
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
import websocket
import datetime
import hashlib
import base64
import hmac
import json
from urllib.parse import urlencode
import time
import ssl
from wsgiref.handlers import format_date_time
from datetime import datetime
from time import mktime
import _thread as thread
import os


STATUS_FIRST_FRAME = 0  # 第一帧的标识
STATUS_CONTINUE_FRAME = 1  # 中间帧标识
STATUS_LAST_FRAME = 2  # 最后一帧的标识


class Ws_Param(object):
    # 初始化
    def __init__(self, APPID, APIKey, APISecret, vcn,Text):
        self.APPID = APPID
        self.APIKey = APIKey
        self.APISecret = APISecret
        self.Text = Text

        # 公共参数(common)
        self.CommonArgs = {"app_id": self.APPID}
        # 业务参数(business)，更多个性化参数可在官网查看
        self.BusinessArgs = {"aue": "lame", "sfl":1,"auf": "audio/L16;rate=16000", "vcn": vcn, "tte": "utf8"}
        self.Data = {"status": 2, "text": str(base64.b64encode(self.Text.encode('utf-8')), "UTF8")}
        #使用小语种须使用以下方式，此处的unicode指的是 utf16小端的编码方式，即"UTF-16LE"”
        #self.Data = {"status": 2, "text": str(base64.b64encode(self.Text.encode('utf-16')), "UTF8")}

    # 生成url
    def create_url(self):
        url = 'wss://tts-api.xfyun.cn/v2/tts'
        # 生成RFC1123格式的时间戳
        now = datetime.now()
        date = format_date_time(mktime(now.timetuple()))

        # 拼接字符串
        signature_origin = "host: " + "ws-api.xfyun.cn" + "\n"
        signature_origin += "date: " + date + "\n"
        signature_origin += "GET " + "/v2/tts " + "HTTP/1.1"
        # 进行hmac-sha256进行加密
        signature_sha = hmac.new(self.APISecret.encode('utf-8'), signature_origin.encode('utf-8'),
                                 digestmod=hashlib.sha256).digest()
        signature_sha = base64.b64encode(signature_sha).decode(encoding='utf-8')

        authorization_origin = "api_key=\"%s\", algorithm=\"%s\", headers=\"%s\", signature=\"%s\"" % (
            self.APIKey, "hmac-sha256", "host date request-line", signature_sha)
        authorization = base64.b64encode(authorization_origin.encode('utf-8')).decode(encoding='utf-8')
        # 将请求的鉴权参数组合为字典
        v = {
            "authorization": authorization,
            "date": date,
            "host": "ws-api.xfyun.cn"
        }
        # 拼接鉴权参数，生成url
        url = url + '?' + urlencode(v)
        # print("date: ",date)
        # print("v: ",v)
        # 此处打印出建立连接时候的url,参考本demo的时候可取消上方打印的注释，比对相同参数时生成的url与自己代码生成的url是否一致
        # print('websocket url :', url)
        return url

def on_message(ws, message):
    try:
       
        message =json.loads(message)
        code = message["code"]
        sid = message["sid"]
        audio = message["data"]["audio"]
        audio = base64.b64decode(audio)
        status = message["data"]["status"]
#         print(message)
        if status == 2:
            print("ws is closed")
            ws.close()
        if code != 0:
            errMsg = message["message"]
            print("sid:%s call error:%s code is:%s" % (sid, errMsg, code))
        else:
            
            with open('./demo.mp3', 'ab') as f:
                f.write(audio)
            
        
    except Exception as e:
        print("receive msg,but parse exception:", e)



# 收到websocket错误的处理
def on_error(ws, error):
    print("### error:", error)


# 收到websocket关闭的处理
def on_close(ws):
    print("### closed ###")


# 收到websocket连接建立的处理
def on_open(ws):
    def run(*args):
        d = {"common": wsParam.CommonArgs,
             "business": wsParam.BusinessArgs,
             "data": wsParam.Data,
             }
        d = json.dumps(d)
        print("------>开始发送文本数据")
        ws.send(d)
        if os.path.exists('./demo.mp3'):
            os.remove('./demo.mp3')

    thread.start_new_thread(run, ())


if __name__ == "__main__":
#     xiaoyan
#     aisjiuxu
#     aisxping
#     aisjinger
#     aisbabyxu
    
    
    t='''
    孩子的愿望和需要是不同的。
    所有孩子真正的需要都应该被满足。
    但是，
    当你屈从于孩子的所有愿望时，
    你就会给孩子和自己造成大问题。
    从降临到你们家的第一天开始，
    你的孩子就有四种基本需要：
    1.归属感和价值感
    2.对自己能力的感知
    3.个人的力量和自主
    4.社会和人生技能
    
    '''
    
    # 测试时候在此处正确填写相关信息即可运行
    wsParam = Ws_Param(APPID='5ef1747b', APIKey='7fdb0bb1237ba905cb0af9232aae6f4a',
                       APISecret='acb36bc820fc089c7e6a7364007f5c22',
                       vcn='xiaoyan',Text=t)
#     websocket.enableTrace(False)
    wsUrl = wsParam.create_url()
    print('正在生成mp3……')
    ws = websocket.WebSocketApp(wsUrl, on_message=on_message, on_error=on_error, on_close=on_close)
    ws.on_open = on_open
    ws.run_forever(sslopt={"cert_reqs": ssl.CERT_NONE})
    print('完成')


正在生成mp3……
------>开始发送文本数据
ws is closed
完成
